In [1]:
import os
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import Model
from keras.utils import multi_gpu_model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dropout,GlobalAveragePooling2D,Dense
from keras import applications as apps
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler,TensorBoard
import keras

Using TensorFlow backend.


In [2]:
path = '/home/ankan/Projects/sleeve/weights/sleeves'

In [3]:
if not os.path.exists(path):
    os.makedirs(path)

In [4]:
labels = os.listdir('/home/ankan/Projects/sleeve/data_sleeves/Resized_Data/Data/train/')
DATA_DIR1 = '/home/ankan/Projects/sleeve/data_sleeves/Resized_Data/Data/'
train_dir = os.path.join(DATA_DIR1, 'train')
test_dir = os.path.join(DATA_DIR1, 'validation')

In [5]:
ft_NB_EPOCHS = 30

In [6]:
def get_nb_files(directory):
  """Get number of files by searching directory recursively"""
  if not os.path.exists(directory):
    return 0
  cnt = 0
  for r, dirs, files in os.walk(directory):
    for dr in dirs:
      cnt += len(glob.glob(os.path.join(r, dr + "/*")))
  return cnt

In [7]:
class LossLearningRateScheduler(keras.callbacks.History):

    def __init__(self, base_lr, lookback_epochs, spike_epochs = None, spike_multiple = 10, decay_threshold = 0.002, decay_multiple = 0.5, loss_type = 'val_loss'):

        super(LossLearningRateScheduler, self).__init__()

        self.base_lr = base_lr
        self.lookback_epochs = lookback_epochs
        self.spike_epochs = spike_epochs
        self.spike_multiple = spike_multiple
        self.decay_threshold = decay_threshold
        self.decay_multiple = decay_multiple
        self.loss_type = loss_type


    def on_epoch_begin(self, epoch, logs=None):
        
        if len(self.epoch) > self.lookback_epochs:

            current_lr = K.get_value(self.model.optimizer.lr)

            target_loss = self.history[self.loss_type] 

            print('loss1: ',target_loss[-int(self.lookback_epochs)])
            print('loss2: ',target_loss[-1])
            

            loss_diff =  target_loss[-int(self.lookback_epochs)] - target_loss[-1]
            
            print('loss1-loss2: ',loss_diff)

            if loss_diff <= np.abs(target_loss[-1]) * (self.decay_threshold * self.lookback_epochs):

                print(' '.join(('Changing learning rate from', str(current_lr), 'to', str(current_lr * self.decay_multiple))))
                K.set_value(self.model.optimizer.lr, current_lr * self.decay_multiple)
                current_lr = current_lr * self.decay_multiple

            else:

                print(' '.join(('Learning rate:', str(current_lr))))

            if self.spike_epochs is not None and len(self.epoch) in self.spike_epochs:
                print(' '.join(('Spiking learning rate from', str(current_lr), 'to', str(current_lr * self.spike_multiple))))
                K.set_value(self.model.optimizer.lr, current_lr * self.spike_multiple)

        else:

            print(' '.join(('Setting learning rate to', str(self.base_lr))))
            K.set_value(self.model.optimizer.lr, self.base_lr)


        return K.get_value(self.model.optimizer.lr)

In [8]:
def main():
    batch_size = 64
    num_classes = len(glob.glob(train_dir + "/*"))
    steps_train = (get_nb_files(train_dir)/batch_size)
    steps_test = (get_nb_files(test_dir)/batch_size)

    base_model = apps.resnet50.ResNet50(weights='imagenet',input_shape=(224,224,3),include_top=False)
    ppf = apps.resnet50.preprocess_input
    
    x = base_model.output
    x = Dropout(0.5)(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024,activation='relu')(x)
    pred = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=pred)

    train_gen = ImageDataGenerator(
        preprocessing_function=ppf)

    train_gen = train_gen.flow_from_directory(
        train_dir,
        target_size=(224,224),
        class_mode='categorical',
        batch_size=batch_size
    )

    test_gen = ImageDataGenerator(
        preprocessing_function=ppf)

    test_gen = test_gen.flow_from_directory(
        test_dir,
        target_size=(224,224),
        class_mode='categorical',
        batch_size=batch_size
    )
    for i, layer in enumerate(base_model.layers):
        layer.trainable = False 
    
    model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
    mod_wt_path = path+'/best_weights.hdf5'
    best_wts_callback = keras.callbacks.ModelCheckpoint(mod_wt_path, save_weights_only=True, save_best_only=True)
    history2 = model.fit_generator(train_gen,steps_per_epoch=steps_train,
                               epochs=ft_NB_EPOCHS,validation_data=test_gen,
                               validation_steps=steps_test,class_weight='auto',callbacks=[best_wts_callback,LossLearningRateScheduler(base_lr = 0.0001, lookback_epochs = 3)])
    model.load_weights(mod_wt_path)
    model.save(path+'/multigpu_128batchsize5stagefull.hdf5')
    
    acc_history = history2.history['acc']
    numpy_acc_history = np.array(acc_history)

    val_acc_history = history2.history['val_acc']
    numpy_loss_history = np.array(val_acc_history)

    loss_history = history2.history['loss']
    numpy_history3 = np.array(loss_history)

    val_loss_history = history2.history['val_loss']
    numpy_history4 = np.array(val_loss_history)

    df = pd.DataFrame(index=range(len(numpy_acc_history)))
    df.loc[:,'acc'] = numpy_acc_history
    df.loc[:,'val_acc'] = numpy_loss_history
    df.loc[:,'loss'] = numpy_history3
    df.loc[:,'val_loss'] = numpy_history4
    df.to_csv(path+"/crop_avgfn_history_fn1_2004"+".csv")

In [9]:
if __name__ == "__main__":
    main()

Found 1221 images belonging to 4 classes.
Found 152 images belonging to 4 classes.
Epoch 1/30
Setting learning rate to 0.0001
20/19 [===============================] - 480s 24s/step - loss: 1.3938 - acc: 0.3878 - val_loss: 1.1583 - val_acc: 0.5000
Epoch 2/30
Setting learning rate to 0.0001
20/19 [===============================] - 456s 23s/step - loss: 1.0443 - acc: 0.5524 - val_loss: 1.0002 - val_acc: 0.5789
Epoch 3/30
Setting learning rate to 0.0001
20/19 [===============================] - 432s 22s/step - loss: 0.8597 - acc: 0.6592 - val_loss: 0.8910 - val_acc: 0.6645
Epoch 4/30
Setting learning rate to 0.0001
20/19 [===============================] - 452s 23s/step - loss: 0.7786 - acc: 0.6919 - val_loss: 0.8290 - val_acc: 0.6250
Epoch 5/30
loss1:  1.0002056454357349
loss2:  0.8290185677377802
loss1-loss2:  0.17118707769795471
Learning rate: 1e-04
20/19 [===============================] - 455s 23s/step - loss: 0.7161 - acc: 0.7306 - val_loss: 0.8646 - val_acc: 0.5658
Epoch 6/30
loss